In [ ]:
%pip install -U pip
%pip install -U botocore boto3
%pip install boto3 anthropic langchain

## 以下のコマンドで最新のmoduleを利用していることを確認してください
- boto3 >= 1.28.57
- botocore >= 1.31.57

In [ ]:
%pip list | grep -E 'botocore|boto3' | awk '{print $1, $2}'

## モデルを利用可能な状態に設定
はじめてそのアカウント、リージョンでBedrockをご利用になる場合は、マネジメントコンソールのModel Accessページからモデルを利用可能にする設定が必要です。

詳細はドキュメントをご確認ください。https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html#add-model-access

## シンプルなBedrockの利用例

In [1]:
import boto3
import json

# Bedrockでは、InvokeModelとInvokeModelWithStreamingResponse APIを呼び出す場合のみ "bedrock-runtime" を利用し、それ以外では" bedrock" を利用します。
bedrock_runtime_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

text = "こんにちは"
# Anthropic社のClaudeモデルでは、以下のようなフォーマットを利用するよう公式サイトに案内があります https://docs.anthropic.com/claude/docs/introduction-to-prompt-design#human--assistant-formatting
# Bedrockではフォーマットに従わない場合エラーが返される挙動になっていますのでご注意下さい。
prompt = f"\n\nHuman: {text}\n\nAssistant:"

response = bedrock_runtime_client.invoke_model(
        body=json.dumps({"prompt": prompt, "max_tokens_to_sample": 100}), modelId="anthropic.claude-v2"
)

response_body = json.loads(response.get("body").read())
print(response_body.get("completion"))



 はい、こんにちは。どうぞよろしくお願いします。


In [2]:
# 利用可能なモデルの確認
bedrock_client = boto3.client(
    service_name="bedrock",
    region_name="us-east-1"
)

for model in bedrock_client.list_foundation_models()["modelSummaries"]:
    print(model["modelId"])

amazon.titan-tg1-large
amazon.titan-e1t-medium
amazon.titan-embed-g1-text-02
amazon.titan-text-lite-v1
amazon.titan-text-express-v1
amazon.titan-embed-text-v1
stability.stable-diffusion-xl
stability.stable-diffusion-xl-v0
ai21.j2-grande-instruct
ai21.j2-jumbo-instruct
ai21.j2-mid
ai21.j2-mid-v1
ai21.j2-ultra
ai21.j2-ultra-v1
anthropic.claude-instant-v1
anthropic.claude-v1
anthropic.claude-v2
cohere.command-text-v14


# Claudeの利用例
- 公式のClaude Prompt Designガイド: https://docs.anthropic.com/claude/docs/introduction-to-prompt-design
- 2023/11/14現在では、Claude Instant v1が東京リージョンで利用可能です。情報抽出・分類等であればぜひ東京リージョンのClaude Instant v1もご利用下さい。

In [3]:
import json
import os
import sys
import time
import boto3
from botocore.session import Session
from botocore.config import Config
from botocore.exceptions import ReadTimeoutError, ClientError
from anthropic import Anthropic


def generate_anthropic(text, model, region):
    # Pricingは変更される可能性がありますので、公式の料金ページを正としてください。 https://aws.amazon.com/jp/bedrock/pricing/
    if model == "claude-v2":
        prompt_cost_per_token = 0.00001102
        completion_cost_per_token = 0.00003268
        model_id="anthropic.claude-v2" 
    elif model == "claude-v1":
        prompt_cost_per_token = 0.00001102
        completion_cost_per_token = 0.00003268
        model_id="anthropic.claude-v1" 
    elif model == "claude-instant-v1":
        prompt_cost_per_token = 0.00000163
        completion_cost_per_token = 0.00000551
        model_id="anthropic.claude-instant-v1" 
    else:
        raise Exception("this model name does not supported here: " + model)

    s = Session()
    boto3_bedrock =  s.create_client(
        service_name="bedrock-runtime",
        region_name=region, 
        config=Config(connect_timeout=300, read_timeout=300, retries={'max_attempts': 1})
    )

    # Claudeでのプロンプトのフォーマットはこちらに。https://docs.anthropic.com/claude/docs/introduction-to-prompt-design#human--assistant-formatting
    # Bedrockではフォーマットに従わない場合エラーが返される挙動になっていますのでご注意下さい。
    prompt = f"{text}"
    # パラメータの詳細はこちらに。https://docs.anthropic.com/claude/reference/complete_post
    # 出力のトークン数の最大数はmax_tokens_to_sampleで変更できます。
    # stop_sequencesには今回は</result>も含めています。
    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 1000, "temperature": 0, "stop_sequences": ["\n\nHuman", "</result>"],
})
    modelId = model_id 
    accept = "application/json"
    contentType = "application/json"
    response = None

    start_time = time.time()
    try:
        response = boto3_bedrock.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
    except ReadTimeoutError as e:
        raise Exception("Timeout occurred while accessing Bedrock. Please increase connect_timeout or read_timeout value in initializing boto3_bedrock.")
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ThrottlingException':
            raise Exception("ThrottlingException occurred. Please note that 'Tokens processed per minute' on claude v2 is limited to 200,000. Ref: https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html")
        raise e
    response_time = (time.time() - start_time) * 1000
    
    response_body = json.loads(response.get("body").read())
    completion = response_body.get("completion")
    
    print(completion)

    # ここではClaudeを利用した際のtoken数、料金、レスポンスタイムを測定しています。

    anthropic_client = Anthropic()
    prompt_tokens = anthropic_client.count_tokens(prompt)
    completion_tokens = anthropic_client.count_tokens(completion)

    prompt_cost = prompt_cost_per_token * prompt_tokens
    completion_cost = completion_cost_per_token * completion_tokens
    total_cost = prompt_cost + completion_cost

    print(f'prompt_tokens: {prompt_tokens:,}, completion_tokens: {completion_tokens:,}, prompt_cost: ${prompt_cost:.9f}, completion_cost: ${completion_cost:.9f}, total_cost: ${total_cost:.9f}')
    print(f'response_time: {response_time:,.0f} ms\n')


In [6]:
transcribed_text = """
営業：こんにちは
顧客：こんにちは
営業：明日はどこへ行くのですか？
顧客：明日は渋谷へ行きます。
"""

prompt = f"""\n\nHuman:
営業と顧客の会話記録を<transcription></transcription>タグの中に記します。

<transcription>
{transcribed_text}
</transcription>

会話記録を参考にして、次にユーザーが取ろうとしている行動を<rule></rule>タグ内の条件に従って回答してください。

<rule>
- "顧客"が取ろうとしている行動を回答してください
- 必ず<transcription></transcription>タグに囲まれた会話記録で述べられている内容から回答を考えてください
- 顧客のことを指す際は"お客様"と呼んで下さい
- 回答は<result></result>タグに含めて回答し、<result></result>タグの前後には何も文字を出力しないでください。
</rule>
\n\nAssistant:<result>
"""
# invoke_model()を呼び出す際に、stop_sequencesというパラメータを設定しているのですが、そこに"</result>"も含めているので、</result>の直前で回答は切られます。


generate_anthropic(prompt, 'claude-v2', 'us-east-1') # 北部バージニアリージョンのclaude-v2はmax 100K tokensを処理できます
generate_anthropic(prompt, 'claude-instant-v1', 'ap-northeast-1') # 東京リージョンのclaude-instant-v1はmax 18K tokensに制限されています

お客様は明日渋谷へ行く予定です。

prompt_tokens: 324, completion_tokens: 21, prompt_cost: $0.003570480, completion_cost: $0.000686280, total_cost: $0.004256760
response_time: 2,096 ms

お客様は明日渋谷へ行かれると述べられています。

prompt_tokens: 324, completion_tokens: 27, prompt_cost: $0.000528120, completion_cost: $0.000148770, total_cost: $0.000676890
response_time: 694 ms



In [5]:
prompt = """\n\nHuman:
以下の<comment></comment>で囲まれたコメントの感情を、<rule></rule>に従って分類してください。

<comment>
1. "帰ったら宿題やらなきゃなあ…"
2. "ここのラーメン超うまい"
3. "購入して１年ちょっとで壊れてしまいました。残念です。"
5. "使い方が理解できなかった"
6. "精度が低く使いものにならない"
7. "ぜひ今後も継続して利用したいです"
8. "思ったより丈夫で軽かったので驚きました"
</comment>

<rule>
- 全てのコメントを「ネガティブ: n」「ポジティブ: p」のいずれかに感情分類してください。コメントのidに対してpかnを回答します
- 結果についてはJSON形式で返してください。それ以外の文字列は返却しないでください。
- JSONは<result></result>タグの中に、次のような形式で1行で出力してください。{"1": "p", "2": "n"}
</rule>

\n\nAssistant:<result>
"""
generate_anthropic(prompt, 'claude-v2', 'us-east-1') # 北部バージニアリージョンのclaude-v2はmax 100K tokensを処理できます
generate_anthropic(prompt, 'claude-instant-v1', 'ap-northeast-1') # 東京リージョンのclaude-instant-v1はmax 18K tokensに制限されています

{"1": "n", "2": "p", "3": "n", "5": "n", "6": "n", "7": "p", "8": "p"}

prompt_tokens: 357, completion_tokens: 43, prompt_cost: $0.003934140, completion_cost: $0.001405240, total_cost: $0.005339380
response_time: 2,416 ms

{"1": "n", "2": "p", "3": "n", "5": "n", "6": "n", "7": "p", "8": "p"}

prompt_tokens: 357, completion_tokens: 43, prompt_cost: $0.000581910, completion_cost: $0.000236930, total_cost: $0.000818840
response_time: 806 ms

